# Project Part 1

[![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/sgeinitz/cs39aa_project/blob/main/project_part1.ipynb)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sgeinitz/cs39aa_project/blob/main/project_part1.ipynb)



## Introduction/Background

In this project I will attempting to train a model to predict news media bias. In order to do this I will be utilizing the dataset NLPCSS-20 from the Association for Computational Linguistics which can be found on Github. The dataset is a collection or corpus contrived of 6,964 news articles. The labels of how we describe the corpus are the following:
- title: title of news
- content: content
- source: source of the content
- allsides-bias: bias (left, center or right)
- misc: aurhor, date, etc
- adfontes_fair: how fair is the news article (bias, neutral or unknown)
- adfontes_political: how political bias skewed the article is (bias, neutral or unknown)
- event_id:event id

This is a text classification problem as how we are trying to classify combination of strings that demonstrate wheter a news content is politically bias or not. The combination of strings is the content itself, it is then labeled with how bias that content is. 


## Exploratory Data Analysis

_You will now load the dataset and carry out some exploratory data analysis steps to better understand what text data looks like. See the examples from class on 10/. The following links provide some good resources of exploratory analyses of text data with Python._


* https://neptune.ai/blog/exploratory-data-analysis-natural-language-processing-tools
* https://regenerativetoday.com/exploratory-data-analysis-of-text-data-including-visualization-and-sentiment-analysis/
* https://medium.com/swlh/text-summarization-guide-exploratory-data-analysis-on-text-data-4e22ce2dd6ad  
* https://www.kdnuggets.com/2019/05/complete-exploratory-data-analysis-visualization-text-data.html  


In [1]:
# import all of the python modules/packages you'll need here
import pandas as pd

print ("hello")
# ...

hello
